# NLP Logistic Regression

[Disaster Tweets Dataset](https://www.kaggle.com/c/nlp-getting-started)

This notebook is inspired by Course 1, Week 1 of the [deeplearning.ai Natural Language Processing Specialization](https://www.deeplearning.ai/natural-language-processing-specialization/), but the code here is implemented using moden library functions rather than using hand-coded implementions.

The approach here is to tokenize the text, and create word frequencies tables for each of the labels. 
A Nx2 numeric feature matrix is created for each tweet, containing the sum of positive and negative frequencies for each word tokens in the tweet.
Linear Regression solves the problem via gradient decent, and is trained to predict labels given an extracted feature matrix.

# Imports

In [ ]:
!pip install -q frozendict > /dev/null

In [ ]:
import numpy  as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import pydash
import math
import os
import itertools

from pydash import flatten, flatten_deep
from collections import Counter, OrderedDict
from frozendict import frozendict
from humanize import intcomma
from operator import itemgetter
from typing import *
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from itertools import product, combinations
from joblib import Parallel, delayed

In [ ]:
df_train = pd.read_csv('../input/nlp-getting-started/train.csv', index_col=0)
df_test  = pd.read_csv('../input/nlp-getting-started/test.csv', index_col=0)
df_train

# Tokenization and Word Frequencies

Here we tokenize the text using nltk.TweetTokenizer, apply lowercasing, tweet preprocessing, and stemming.

Then compute a dictionary lookup of word counts for each label

In [ ]:
print(nltk.corpus.stopwords.words('english'))

In [ ]:
def tokenize_df(
    dfs: List[pd.DataFrame], 
    keys          = ('text', 'keyword', 'location'), 
    stemmer       = True, 
    preserve_case = True, 
    reduce_len    = False, 
    strip_handles = True,
    use_stopwords = True,
    **kwargs,
) -> List[List[str]]:
    # tokenizer = nltk.TweetTokenizer(preserve_case=True,  reduce_len=False, strip_handles=False)  # defaults 
    tokenizer = nltk.TweetTokenizer(preserve_case=preserve_case, reduce_len=reduce_len, strip_handles=strip_handles) 
    porter    = nltk.PorterStemmer()
    stopwords = set(nltk.corpus.stopwords.words('english') + [ 'nan' ])

    output    = []
    for df in flatten([ dfs ]):
        for index, row in df.iterrows():
            tokens = flatten([
                tokenizer.tokenize(str(row[key] or ""))
                for key in keys    
            ])
            if use_stopwords:
                tokens = [ 
                    token 
                    for token in tokens 
                    if token.lower() not in stopwords
                    and len(token) >= 2
                ]                
            if stemmer:
                tokens = [ 
                    porter.stem(token) 
                    for token in tokens 
                ]
            output.append(tokens)

    return output


def word_frequencies(df, **kwargs) -> Dict[int, Counter]:
    tokens = {
        0: flatten(tokenize_df( df[df['target'] == 0], **kwargs )),
        1: flatten(tokenize_df( df[df['target'] == 1], **kwargs )),
    }
    freqs = { 
        target: Counter(dict(Counter(tokens[target]).most_common())) 
        for target in [0, 1]
    }  # sort and cast
    return freqs

In [ ]:
tokenize_df(df_train)[:2]

In [ ]:
freqs = word_frequencies(df_train)
print('freqs[0]', len(freqs[0]), freqs[0].most_common(10))
print('freqs[1]', len(freqs[1]), freqs[1].most_common(10))

# Feature Extraction

Here we create a Nx2 feature matrix containing the sum of positive and negative word frequencies for each tweet

In [ ]:
def inverse_document_frequency( tokens: List[str] ) -> Counter:
    tokens = flatten_deep(tokens)
    idf = {
        token: math.log( len(tokens) / count ) 
        for token, count in Counter(tokens).items()
    }
    idf = Counter(dict(Counter(idf).most_common()))  # sort and cast
    return idf

def inverse_document_frequency_df( dfs ) -> Counter:
    tokens = flatten_deep([ tokenize_df(df) for df in flatten([ dfs ]) ])
    return inverse_document_frequency(tokens)

idf = inverse_document_frequency_df([ df_train, df_test ])
list(reversed(idf.most_common()))[:20]

In [ ]:
def extract_features(df, freqs, use_idf=True, use_log=True, **kwargs) -> np.array:
    features = []
    tokens   = tokenize_df(df, **kwargs)
    for n in range(len(tokens)):
        bias     = 1  # bias term is implict when using sklearn
        positive = 1
        negative = 1        
        for token in tokens[n]:
            if use_idf:
                positive += freqs[0].get(token, 0) * idf.get(token, 1) 
                negative += freqs[1].get(token, 0) * idf.get(token, 1)
            else:
                positive += freqs[0].get(token, 0) 
                negative += freqs[1].get(token, 0) 
        features.append([ positive, negative ])  

    features = np.array(features)   # accuracy = 0.7166688559043741
    if use_log:
        features = np.log(features) # accuracy = 0.7136477078681204
    return features


Y_train = df_train['target'].to_numpy()
X_train = extract_features(df_train, freqs)
X_test  = extract_features(df_test,  freqs)

print('df_train', df_train.shape)
print('df_test ', df_test.shape)
print('Y_train ', Y_train.shape)
print('X_train ', X_train.shape)
print('X_test  ', X_test.shape)
print(X_test[:5])

# Hyperparameter Search


The optimal settings are:
- stemmer = True
- preserve_case = True
- strip_handles = Any
- use_stopwords = True

The above are exactly opposite compared to [TF-IDF Classifier](https://www.kaggle.com/jamesmcguigan/disaster-tweets-tf-idf-classifier?scriptVersionId=50898834), 
but the following settings are shared:

- use_idf = True
- use_log = True

In [ ]:
def predict_df(df_train, df_test, **kwargs):
    freqs   = word_frequencies(df_train, **kwargs)

    Y_train = df_train['target'].to_numpy()
    X_train = extract_features(df_train, freqs, **kwargs)
    X_test  = extract_features(df_test,  freqs, **kwargs) \
              if df_train is not df_test else X_train

    model      = LinearRegression().fit(X_train, Y_train)
    prediction = model.predict(X_test)
    prediction = np.round(prediction).astype(np.int)
    return prediction


def get_train_accuracy(splits=3, **kwargs):
    """ K-Fold Split Accuracy """
    accuracy = 0.0
    for _ in range(splits):
        train, test = train_test_split(df_train, test_size=1/splits)      
        prediction  = predict_df(train, test, **kwargs)
        Y_train     = test['target'].to_numpy()
        accuracy   += np.sum( Y_train == prediction ) / len(Y_train) / splits    
    return accuracy
    
    
def train_accuracy_hyperparameter_search():
    results = Counter()
    jobs    = []
    
    # NOTE: reducing input fields has no effect on accuracy
    for keys in [ ('text', 'keyword', 'location'), ]: # ('text', 'keyword'), ('text',) ]:
        strip_handles = 1  # no effect on accuracy 
        # use_log       = 1  # no effect on accuracy
        for stemmer, preserve_case, reduce_len, use_stopwords, use_idf, use_log in product([1,0],[1,0],[1,0],[1,0],[1,0],[1,0]):
            def fn(keys, stemmer, preserve_case, reduce_len, strip_handles, use_stopwords, use_idf, use_log):
                kwargs = {
                    "stemmer":        stemmer,          # stemmer = True is always better
                    "preserve_case":  preserve_case, 
                    "reduce_len":     reduce_len, 
                    # "strip_handles": strip_handles,   # no effect on accuracy
                    "use_stopwords":  use_stopwords,    # use_stopwords = True is always better
                    "use_idf":        use_idf,          # use_idf = True is always better
                    "use_log":        use_log,          # use_log = True is always better
                }
                label = frozendict({
                    **kwargs,
                    # "keys": keys,                     # no effect on accuracy
                })
                accuracy = get_train_accuracy(**kwargs)
                return (label, accuracy)
            
            # hyperparameter search is slow, so multiprocess it
            jobs.append( delayed(fn)(keys, stemmer, preserve_case, reduce_len, strip_handles, use_stopwords, use_idf, use_log) )
            
    results = Counter(dict( Parallel(-1)(jobs) ))
    results = Counter(dict(results.most_common()))  # sort and cast
    return results

In [ ]:
%%time
if True or os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'Localhost') == 'Batch':
    results = train_accuracy_hyperparameter_search()
    for label, value in results.items():
        print(f'{value:.5f} |', "  ".join(f"{k.split('_')[-1]} = {v}" for k,v in label.items() ))  # pretty printdd

Verify train accuracy given default settings

In [ ]:
print('train_accuracy = ', get_train_accuracy())

# Submission

Without additional feature engineering, LinearRegression scores worst than my [TF-IDF Classifier](https://www.kaggle.com/jamesmcguigan/disaster-tweets-tf-idf-classifier?scriptVersionId=50898834)

In [ ]:
df_submission = pd.DataFrame({
    "id":     df_test.index,
    "target": predict_df(df_train, df_test)
})
df_submission.to_csv('submission.csv', index=False)
! head submission.csv

# Further Reading

This notebook is part of a series exploring Natural Language Processing
- 0.74164 - [NLP Logistic Regression](https://www.kaggle.com/jamesmcguigan/disaster-tweets-logistic-regression/)
- 0.77536 - [NLP TF-IDF Classifier](https://www.kaggle.com/jamesmcguigan/disaster-tweets-tf-idf-classifier)
- 0.79742 - [NLP Naive Bayes](https://www.kaggle.com/jamesmcguigan/nlp-naive-bayes)